In [1]:
import imaplib, email, random, re

In [2]:
nonos = ['http', '<', '{', '(', '\\', '[', ')', '[']

def email_cleaner(text):
    text = re.sub('\W+',' ',text).strip()
    text = text.replace('\n', '')
    text = text.replace('\r', ' ')
    subs = text.split(' ')
    subs = list(filter(('').__ne__, subs))
    
    to_remove = []
    for token in subs : 
        flag = True 
        for nono in nonos : 
            if nono in token : 
                flag = False 
                break 
        if not flag : 
            to_remove.append(token)

    output = [i for i in subs if i not in to_remove]
    text = ' '.join(output)
    return text 

In [60]:
host = 'imap.gmail.com'
spam_folder_name = "\"" + "Collegeboard spam" + "\""

with open('/users/gursi/desktop/gmail_login.txt', 'r') as f : 
    username = f.readline().replace('\n','')
    password = f.readline().replace('\n','')

mail = imaplib.IMAP4_SSL(host)
mail.login(username, password)
mail.select(spam_folder_name, readonly=False)
# mail.select("inbox", readonly=False)
print('Logged in.')

_, search_data = mail.search(None, 'UNSEEN')
mails = search_data[0].split()

for uid in mails:
    _, data = mail.fetch(uid, '(RFC822)')
    _, b = data[0]
    email_msg = email.message_from_bytes(b)
    print(f'uid: {uid}')
    for header in ['subject', 'to', 'from', 'date']:
        print(f'{header}: {email_msg[header]}')
    for part in email_msg.walk():
        if part.get_content_type()=='text/plain' :
            body = part.get_payload(decode=True).decode()
            body = email_cleaner(body)
            print(body)
            with open('/users/gursi/desktop/email.txt', 'w') as f:
                f.writelines([body])
    break

Logged in.
uid: b'3982'
subject: Sample email for relabelling and deletion
to: gursimar.batra@gmail.com
from: Gursimar Singh <gursiml2602@gmail.com>
date: Sun, 26 Sep 2021 11:15:39 +0800
Sample email for relabelling and deletion


In [61]:
def mark_unseen(mail_uid):
    r = mail.store(mail_uid, '-FLAGS', '\Seen')
    if r[0] == 'OK':
        print('Mail marked as unseen.')

def relabel_and_delete(mail_uid, spam_folder_name):
    r1 = mail.store(mail_uid, '+X-GM-LABELS', spam_folder_name)
    mark_unseen(mail_uid)
    if r1[0] == 'OK':
        print('Mail relabeled.')
        r3 = mail.store(mail_uid, "+FLAGS", "\\Deleted")
        if r3[0] == 'OK' : 
            print('Mail deleted.')

In [62]:
relabel_and_delete(uid, spam_folder_name)

Mail marked as unseen.
Mail relabeled.
Mail deleted.
